## Remove silence from original audios

In [1]:
%%capture
pip install numpy pandas pydub scipy sounddevice soundfile

In [3]:
import os
import warnings

import numpy as np
import pandas as pd 

import sounddevice as sd
import soundfile as sf
from pydub import AudioSegment
from pydub.silence import split_on_silence

warnings.filterwarnings("ignore", category=DeprecationWarning)

AUDIO_FORMAT = "WAV"

/root/venv/lib/python3.9/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


## Train

In [4]:
train_audio_path = '/work/deep_learning/project_2/train/audio'
train_audio_path_without_silence = '/work/deep_learning/project_2/train/audio_without_silence'

class_names = []

for file in os.listdir(train_audio_path_without_silence):
    d = os.path.join(train_audio_path_without_silence, file)
    if os.path.isdir(d):
        class_names.append(str(os.path.basename(d)))

In [7]:
for class_name in class_names:
    class_path = f"{train_audio_path}/{class_name}"
    class_path_without_silence = f"{train_audio_path_without_silence}/{class_name}"

    if not os.path.exists(class_path_without_silence):
        os.makedirs(class_path_without_silence)
    
    class_files = [f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))]
    for class_file in class_files:

        # remove silence  
        sound = AudioSegment.from_file(f"{class_path}/{class_file}", format=AUDIO_FORMAT) 
        audio_chunks = split_on_silence(
            sound,
            min_silence_len = 100,
            silence_thresh = -45,
            keep_silence = 50
        )
        combined = AudioSegment.empty()
        for chunk in audio_chunks:
            combined += chunk

        combined.export(f"{class_path_without_silence}/{class_file}", format=AUDIO_FORMAT)

### Test of silence removal (works only on computer)

#### Before

In [ ]:
data, fs = sf.read("train/audio/right/0a7c2a8d_nohash_0.wav", dtype='float32')  
sd.play(data, fs)
status = sd.wait()

#### After


In [ ]:
data, fs = sf.read("train/audio_without_silence/right/0a7c2a8d_nohash_0.wav", dtype='float32')  
sd.play(data, fs)
status = sd.wait()

## Test

In [ ]:
train_audio_path = 'test/audio'
train_audio_path_without_silence = 'test/audio_without_silence'


In [ ]:
files = [f for f in os.listdir(train_audio_path) if os.path.isfile(os.path.join(train_audio_path, f))]
for file in files:

    # remove silence  
    sound = AudioSegment.from_file(f"{train_audio_path}/{file}", format=AUDIO_FORMAT) 
    
    if sound.dBFS >= -45:
        audio_chunks = split_on_silence(
            sound,
            min_silence_len = 100,
            silence_thresh = -45,
            keep_silence = 50
        )

    combined = AudioSegment.empty()
    for chunk in audio_chunks:
        combined += chunk

    combined.export(f"{train_audio_path_without_silence}/{file}", format=AUDIO_FORMAT)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6ec8773e-c89e-4114-ab03-f7a026dc9b15' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>